In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

# 读取数据
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:
# Swiss army knife function to organize the data

# 提取label，并编码成数字
def encode(train, test):
    le = LabelEncoder().fit(train.species) 
    labels = le.transform(train.species)           # encode species strings
    classes = list(le.classes_)                    # save column names for submission
    test_ids = test.id                             # save test ids for submission
    
    train = train.drop(['species', 'id'], axis=1)  
    test = test.drop(['id'], axis=1)
    
    train = StandardScaler().fit(train.values).transform(train.values)
    test = StandardScaler().fit(test.values).transform(test.values)
    
    return train, labels, test, test_ids, classes

train, labels, test, test_ids, classes = encode(train, test)

In [3]:
# 在train数据里划分训练测试数据
sss = StratifiedShuffleSplit(labels, 10, test_size=0.2, random_state=23)

for train_index, test_index in sss:
    X_train, X_test = train[train_index], train[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

In [4]:
?tf.contrib.learn.DNNClassifier()

In [5]:
dnn_classifier = tf.contrib.learn.DNNClassifier(hidden_units=[1024, 512, 256],n_classes=99, dropout=0.2)
#dnn_classifier = tf.contrib.learn.DNNLinearCombinedClassifier(dnn_hidden_units=[1024, 512, 256],n_classes=99)

Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.


In [6]:
dnn_classifier.fit(x=X_train, y=y_train, steps=8000)

DNNClassifier(hidden_units=[1024, 512, 256], dropout=0.2, optimizer=None, feature_columns=None)

In [ ]:
dnn_results = dnn_classifier.evaluate(x=X_test, y=y_test, steps=1)

In [ ]:
for key in sorted(dnn_results):
    print "%s: %s" %(key, dnn_results[key])

In [ ]:
#linear_classifier = tf.contrib.learn.LinearClassifier(n_classes=99)

In [ ]:
#?tf.contrib.learn.LinearClassifier

In [ ]:
#linear_classifier.fit(x=X_train, y=y_train, steps=10000)

In [ ]:
#lin_results = linear_classifier.evaluate(x=X_test, y=y_test, steps=1)

In [ ]:
#for key in sorted(lin_results):
#    print "%s: %s" %(key, lin_results[key])

In [ ]:
dnn_classifier.fit(x=train, y=labels, steps=8000)

In [ ]:
#linear_classifier.fit(x=train, y=labels, steps=10000)

In [ ]:
y_dnn_prob = dnn_classifier.predict_proba(test)
#y_lin_prob = linear_classifier.predict_proba(test)

In [ ]:
#y_prob = 0.5*y_dnn_prob + 0.5*y_lin_prob

In [ ]:
#def combine(x, y):
#    if x < 0.5 and y<0.5:
#        return min(x,y)
#    elif x>0.5 and y>0.5:
#        return max(x,y)
#    else:
#        return 0.5*x + 0.5*y

In [ ]:
#y = combine(0.6, 0.7)
#y

In [ ]:
#y_prob = np.array([[combine(a,b) for a,b in zip(x,y)] for x,y in zip(y_dnn_prob, y_lin_prob)])
#y_prob

In [ ]:
#dnn_submission = pd.DataFrame(y_dnn_prob, index=test_ids, columns=classes)
submission = pd.DataFrame(y_dnn_prob, index=test_ids, columns=classes)

In [ ]:
#dnn_submission = dnn_submission.reset_index()
submission = submission.reset_index()

In [ ]:
submission.head()

In [ ]:
#dnn_submission.to_csv("dnn_submission.csv", index=False)
submission.to_csv("dnn_submission.csv", index=False)